In [7]:
import h5py
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [8]:
!unzip "/content/gdrive/My Drive/datasets/data.h5.zip" -d "./"

Archive:  /content/gdrive/My Drive/datasets/data.h5.zip
replace ./data1.h5? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


# PART A TASK 1

In [0]:
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import h5py


def loadDataH5():
    with h5py.File('data1.h5','r') as hf:
        trainX = np.array(hf.get('trainX'))
        trainY = np.array(hf.get('trainY'))
        valX = np.array(hf.get('valX'))
        valY = np.array(hf.get('valY'))
        print (trainX.shape,trainY.shape)
        print (valX.shape,valY.shape)
    return trainX, trainY, valX, valY

In [0]:

class cnn:

  @staticmethod
  def buildBasic(width,height,depth,classes):
    model = tf.keras.Sequential()
    inputshape = (height,width,depth)

    model.add(tf.keras.layers.Conv2D(16, (3,3), padding='same', input_shape=inputshape, activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(500, activation='relu'))
    model.add(tf.keras.layers.Dense(classes, activation='softmax'))

    return model

  @staticmethod
  def buildVersion1(width,height,depth,classes):
    model = tf.keras.Sequential()
    inputshape = (height,width,depth)

    model.add(tf.keras.layers.Conv2D(16, (3,3), padding='same', input_shape=inputshape, activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(tf.keras.layers.Conv2D(32, (3,3), padding='same', activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(64, activation='relu'))
    model.add(tf.keras.layers.Dense(classes, activation='softmax'))

    return model

  @staticmethod
  def buildVersion2(width,height,depth,classes):
    model = tf.keras.Sequential()
    inputshape = (height,width,depth)

    model.add(tf.keras.layers.Conv2D(16, (3,3), padding='same', input_shape=inputshape, activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(tf.keras.layers.Conv2D(32, (3,3), padding='same', activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(64, activation='relu'))
    model.add(tf.keras.layers.Dense(classes, activation='softmax'))

    return model

  @staticmethod
  def buildVersion3(width,height,depth,classes):
    model = tf.keras.Sequential()
    inputshape = (height,width,depth)

    model.add(tf.keras.layers.Conv2D(16, (3,3), padding='same', input_shape=inputshape, activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(tf.keras.layers.Conv2D(32, (3,3), padding='same', activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(tf.keras.layers.Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(64, activation='relu'))
    model.add(tf.keras.layers.Dense(classes, activation='softmax'))

    return model


def task1():
  
  NUM_EPOCHS = 20

  # ======================
  # CHANGE MODEL TYPE HERE
  # ======================
  # basic cnn = 0
  # 1 extra pool and conv layer = 1
  # 2 extra pool and conv layers = 2
  # 3 extra pool and conv layers = 3
  # data augmentation will prompt to enable for the two deepest network
  modelType = 3

  # DATA AUGMENTATION VARIABLES (do not change)
  augmentData = False
  augmentBatch = 8

  # load data
  trainX, trainY, testX, testY = loadDataH5()
  opt = tf.keras.optimizers.SGD(lr=0.01)

  if modelType == 0:
    model = cnn.buildBasic(width=128,height=128,depth=3,classes=17)
  elif modelType == 1:
    model = cnn.buildVersion1(width=128,height=128,depth=3,classes=17)
  elif modelType == 2:
    print("Do you want to do data augmentation for this model? [y = yes] [any key = no]")
    inp = input()
    if inp == 'y' or inp=='Y':
      augmentData = True
    model = cnn.buildVersion2(width=128,height=128,depth=3,classes=17)
  elif modelType == 3:
    print("Do you want to do data augmentation for this model? [y = yes] [any key = no]")
    inp = input()
    if inp == 'y' or inp=='Y':
      augmentData = True
    model = cnn.buildVersion3(width=128,height=128,depth=3,classes=17)
  else:
    print ("wrong model type input")
    quit()

  if augmentData:
    aug= tf.keras.preprocessing.image.ImageDataGenerator( rotation_range=20, width_shift_range=0.1, shear_range=0.2, zoom_range=0.4, horizontal_flip=True)
    trainGenerator= aug.flow(trainX, trainY, batch_size=32)

  model.compile(loss="sparse_categorical_crossentropy",optimizer=opt,metrics=["accuracy"])
  print (model.summary())

  if augmentData:
    H = model.fit(trainGenerator,validation_data=(testX,testY),batch_size=1,epochs=NUM_EPOCHS, steps_per_epoch= len(trainX)/ augmentBatch)
  else:
    H = model.fit(trainX,trainY,validation_data=(testX,testY),batch_size=1,epochs=NUM_EPOCHS)

  print ("Test Data Loss and Accuracy: ", model.evaluate(testX, testY))

  # plot the training loss and accuracy
  plt.style.use("ggplot")
  plt.figure()
  plt.plot(np.arange(0, NUM_EPOCHS), H.history["loss"], label="train_loss")
  plt.plot(np.arange(0, NUM_EPOCHS), H.history["val_loss"], label="val_loss")
  plt.plot(np.arange(0, NUM_EPOCHS), H.history["accuracy"], label="train_acc")
  plt.plot(np.arange(0, NUM_EPOCHS), H.history["val_accuracy"], label="val_acc")
  plt.title("Training Loss and Accuracy")
  plt.xlabel("Epoch #")
  plt.ylabel("Loss/Accuracy")
  plt.legend()
  plt.show()


task1()

# PART A TASK 2

In [10]:
class models:

  def shallowVGG(width,height,depth,classes):
    model = tf.keras.models.Sequential()
    inputShape= (height, width, depth)

    model.add(tf.keras.layers.Conv2D(32, (3, 3), padding="same",input_shape=inputShape, activation='relu'))
    model.add(tf.keras.layers.Conv2D(32, (3, 3), padding="same", activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(tf.keras.layers.Conv2D(64, (3, 3), padding="same", activation='relu'))
    model.add(tf.keras.layers.Conv2D(64, (3, 3), padding="same", activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(512,activation='relu'))
    model.add(tf.keras.layers.Dense(classes, activation='softmax'))

    return model

  def AlexNet(width,height,depth,classes):
    model = tf.keras.models.Sequential()
    inputShape= (height, width, depth)

    model.add(tf.keras.layers.Conv2D(96, (11, 11), strides=4, padding="same",input_shape=inputShape, activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2,2)))

    model.add(tf.keras.layers.Conv2D(256, (5, 5), padding="same", activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2,2)))

    model.add(tf.keras.layers.Conv2D(384, (3, 3), padding="same", activation='relu'))
    model.add(tf.keras.layers.Conv2D(384, (3, 3), padding="same", activation='relu'))
    model.add(tf.keras.layers.Conv2D(256, (3, 3), padding="same", activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2,2)))

    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(9216,activation='relu'))
    model.add(tf.keras.layers.Dense(4096,activation='relu'))
    model.add(tf.keras.layers.Dense(4096,activation='relu'))
    model.add(tf.keras.layers.Dense(classes, activation='softmax'))

    return model

  def LeNet(width,height,depth,classes):

    model = tf.keras.models.Sequential()
    inputShape= (height, width, depth)

    model.add(tf.keras.layers.Conv2D(20, (5, 5), padding="same",input_shape=inputShape, activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2),strides=(2, 2)))

    model.add(tf.keras.layers.Conv2D(50, (5, 5), padding="same", activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2),strides=(2, 2)))

    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(500, activation='relu'))
    model.add(tf.keras.layers.Dense(classes, activation='softmax'))

    return model


def ensemble(modelName, width, height, depth, classes):
  if modelName == "ShallowVGG":
    return models.shallowVGG(width, height, depth, classes)
  elif modelName == "AlexNet":
    return models.AlexNet(width, height, depth, classes)
  elif modelName == "LeNet":
    return models.LeNet(width, height, depth, classes)
  elif modelName == "VGG16":
    return tf.keras.applications.VGG16(weights=None, input_shape=(width, height, depth), classes=classes)
  elif modelName == "Inception":
    return tf.keras.applications.InceptionV3(weights=None, input_shape=(width, height, depth), classes=classes)
  elif modelName == "ResNet":
    return tf.keras.applications.ResNet50(weights=None, input_shape=(width, height, depth), classes=classes)


def task2():

  NUM_EPOCHS = 50
  TRAIN = False # set to TRUE to re-train all base learners to get new weights

  trainX, trainY, testX, testY = loadDataH5()
  opt = tf.keras.optimizers.SGD(lr=0.01)
  augmentBatch = 8

  modelnames = ["ResNet","Inception","VGG16","AlexNet","LeNet","ShallowVGG"]

  # TRAINING FOR ALL BASE LEARNERS
  if TRAIN:
    for name in modelnames:
      if name != "ResNet" or name != "Inception" or name != "VGG16":
        model = ensemble(name,width=128,height=128,depth=3,classes=17)

        # data augmentation
        aug= tf.keras.preprocessing.image.ImageDataGenerator( rotation_range=20, width_shift_range=0.1, shear_range=0.2, zoom_range=0.4, horizontal_flip=True)
        trainGenerator= aug.flow(trainX, trainY, batch_size=32)

        model.compile(loss="sparse_categorical_crossentropy",optimizer=opt,metrics=["accuracy"])
        print (model.summary())

        # initialize checkpoint to monitor best validation loss
        # save weights to drive
        fname = "/content/gdrive/My Drive/Deeplearning/weights/weights."+name+".hdf5"
        checkpoint = tf.keras.callbacks.ModelCheckpoint(fname, monitor="val_loss", mode="min", save_best_only=True, verbose=1)

        H = model.fit(trainGenerator,validation_data=(testX,testY),batch_size=1,epochs=NUM_EPOCHS, steps_per_epoch= len(trainX)/ augmentBatch, callbacks=[checkpoint])

        print ("Test Data Loss and Accuracy: ", model.evaluate(testX, testY))

        # plot the training loss and accuracy
        plt.style.use("ggplot")
        plt.figure()
        plt.plot(np.arange(0, NUM_EPOCHS), H.history["loss"], label="train_loss")
        plt.plot(np.arange(0, NUM_EPOCHS), H.history["val_loss"], label="val_loss")
        plt.plot(np.arange(0, NUM_EPOCHS), H.history["accuracy"], label="train_acc")
        plt.plot(np.arange(0, NUM_EPOCHS), H.history["val_accuracy"], label="val_acc")
        plt.title("Training Loss and Accuracy")
        plt.xlabel("Epoch #")
        plt.ylabel("Loss/Accuracy")
        plt.legend()
        plt.show()

  # PREDICTIONS FOR ALL BASE LEARNERS
  print ("Predicting base learners")
  all_predictions = np.zeros((340,17))
  for name in modelnames:
    model = ensemble(name,width=128,height=128,depth=3,classes=17)

    # load weights from drive
    fname = "/content/gdrive/My Drive/Deeplearning/weights/weights."+name+".hdf5"
    model.load_weights(fname)
    
    opt = tf.keras.optimizers.SGD(lr=0.01)
    model.compile(loss="sparse_categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

    predictions = model.predict(testX)
    print ("Prediction accuracy of", name,":", np.sum(np.argmax(predictions, axis=1) == testY)/len(testY))
    all_predictions += predictions

  # AVERAGING PREDICTIONS TO OBTAIN ENSEMBLE PREDICTIONS ACCURACY
  average = all_predictions / len(modelnames)
  final_pred = np.argmax(average, axis=1)
  print()
  print("Final ensemble accuracy:",np.sum(final_pred == testY)/len(testY))


task2()

(1020, 128, 128, 3) (1020,)
(340, 128, 128, 3) (340,)
Predicting base learners
Prediction accuracy of ResNet : 0.7823529411764706
Prediction accuracy of Inception : 0.8
Prediction accuracy of VGG16 : 0.6676470588235294
Prediction accuracy of AlexNet : 0.7176470588235294
Prediction accuracy of LeNet : 0.6970588235294117
Prediction accuracy of ShallowVGG : 0.7

Final ensemble accuracy: 0.8294117647058824
